In [1]:
from pynq import Overlay, allocate
import numpy as np
import time

# Load the overlay (replace with the correct path if needed)
ol = Overlay("sistema.bit")

In [2]:
# Define the array size: 2^13 = 8192 elements
size = 8192

# Allocate contiguous memory for input and output buffers
input_buffer = allocate(shape=(size,), dtype=np.float32)
output_buffer = allocate(shape=(size,), dtype=np.float32)

# Initialize input data (for example, with random numbers)
input_buffer[:] = np.random.rand(size).astype(np.float32)


gain = np.float32(2.0)  # Set your gain value

In [3]:
# Obtener la instancia del IP (asegúrate de que el nombre coincida con el del diseño)
kr = ol.array_gain_0

# Ver el mapa de registros (opcional)
print("Mapa de Registros:")
print(kr.register_map)

Mapa de Registros:
RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  input_r_1 = Register(input_r=write-only),
  input_r_2 = Register(input_r=write-only),
  output_r_1 = Register(output_r=write-only),
  output_r_2 = Register(output_r=write-only),
  gain = Register(gain=write-only)
}


In [4]:
# Escribir las direcciones físicas y el parámetro a los registros utilizando el mapa:
kr.write(kr.register_map.input_r_1.address, input_buffer.physical_address)
#kr.write(kr.register_map.input_r_2.address, 0)  # Suponemos que la dirección cabe en 32 bits

kr.write(kr.register_map.output_r_1.address, output_buffer.physical_address)
#kr.write(kr.register_map.output_r_2.address, 0)

# Convert the float 'gain' into its 32-bit integer representation and cast to int:
gain_int = int(np.frombuffer(gain.tobytes(), dtype=np.uint32)[0])
kr.write(kr.register_map.gain.address, gain_int)

# Start the kernel by writing '1' to the control register at offset 0x00
kr.register_map.CTRL.AP_START = 1
time.sleep(1)



In [5]:
# Now, output_buffer should contain the results (each element multiplied by 'gain')
print("First 10 input values  :", input_buffer[:10])
print("First 10 output values :", output_buffer[:10])
print("Gain used              :", gain)

First 10 input values  : [0.11859511 0.26785994 0.44953153 0.3354437  0.77005535 0.04303255
 0.99107796 0.76062053 0.5924677  0.76602286]
First 10 output values : [0.23719022 0.5357199  0.89906305 0.6708874  1.5401107  0.08606509
 1.9821559  1.5212411  1.1849355  1.5320457 ]
Gain used              : 2.0


In [6]:
# When finished, free the allocated buffers (if desired)
input_buffer.freebuffer()
output_buffer.freebuffer()